In [25]:
from pose_utils import *
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.metrics import confusion_matrix
import pickle

In [23]:
df_concat=pd.DataFrame()
for i in os.listdir('/mnt/disks/cricket-highlights/anvesh/AlphaPose/output/badm004/'):
    print(i)
    path = '/mnt/disks/cricket-highlights/anvesh/AlphaPose/output/badm004/'+i+'/alphapose-results.json'
    df = pd.read_json(path)
    df['clip']=str(i)
    df['normalized_tuples'] = df.keypoints.apply(normalize_tuples)
    df['normalized_keypoints'] = df.keypoints.apply(normalize_keypoints)
    df['width'] = df.keypoints.apply(get_width)
    df['height'] = df.keypoints.apply(get_height)
    df['angle_at_hip'] = df.keypoints.apply(get_angle_at_hip)
    df['is_person_standing'] = df.keypoints.apply(is_person_standing_straight)
    df_concat = df_concat.append(df)

badminton004_clip_00000519
badminton004_clip_00000409
badminton004_clip_00000003
badminton004_clip_00000338
badminton004_clip_00000155
badminton004_clip_00000430
badminton004_clip_00000177
badminton004_clip_00000164
badminton004_clip_00000521
badminton004_clip_00000365
badminton004_clip_00000187
badminton004_clip_00000474
badminton004_clip_00000234
badminton004_clip_00000130
badminton004_clip_00000279
badminton004_clip_00000370
badminton004_clip_00000334
badminton004_clip_00000214
badminton004_clip_00000359
badminton004_clip_00000271
badminton004_clip_00000136
badminton004_clip_00000250
badminton004_clip_00000291
badminton004_clip_00000392
badminton004_clip_00000399
badminton004_clip_00000204
badminton004_clip_00000327
badminton004_clip_00000197
badminton004_clip_00000493
badminton004_clip_00000241
badminton004_clip_00000088
badminton004_clip_00000048
badminton004_clip_00000397
badminton004_clip_00000367
badminton004_clip_00000313
badminton004_clip_00000195
badminton004_clip_00000039
b

badminton004_clip_00000420
badminton004_clip_00000223
badminton004_clip_00000137
badminton004_clip_00000127
badminton004_clip_00000267
badminton004_clip_00000141
badminton004_clip_00000070
badminton004_clip_00000552
badminton004_clip_00000404
badminton004_clip_00000294
badminton004_clip_00000537
badminton004_clip_00000297
badminton004_clip_00000222
badminton004_clip_00000307
badminton004_clip_00000518
badminton004_clip_00000331
badminton004_clip_00000339
badminton004_clip_00000554
badminton004_clip_00000550
badminton004_clip_00000050
badminton004_clip_00000247
badminton004_clip_00000491
badminton004_clip_00000471
badminton004_clip_00000054
badminton004_clip_00000438
badminton004_clip_00000058
badminton004_clip_00000273
badminton004_clip_00000006
badminton004_clip_00000411
badminton004_clip_00000157
badminton004_clip_00000287
badminton004_clip_00000332
badminton004_clip_00000382
badminton004_clip_00000153
badminton004_clip_00000040
badminton004_clip_00000076
badminton004_clip_00000022
b

In [12]:
res = pd.read_json('../output/badminton004.json')
play_clips = [i.split('\\')[-1].split('.')[0] for i in res.video]

In [13]:
df_concat['play'] = [1 if i in play_clips else 0 for i in df_concat['clip']  ]

In [14]:
# Stores only the person with max height. All other persons in the image are dropped
df_concat= df_concat.sort_values(by=['image_id', 'height'],ascending=False).drop_duplicates(['clip','image_id'])

In [16]:
X_mat = np.vstack( (df_concat.normalized_keypoints))
y= df_concat.play

In [17]:
gss = GroupShuffleSplit(n_splits=10, test_size=0.2, random_state=13)
data_split = gss.split(X_mat, y, groups = df_concat['clip'])
train_ids, test_ids = next(data_split)
X_train, X_test, y_train, y_test = X_mat[train_ids],X_mat[test_ids],np.array(y)[train_ids],np.array(y)[test_ids] 

In [18]:
clf = rfc()
clf.fit(X_train,y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
pred = clf.predict(X_test)

In [22]:
print(confusion_matrix(y_test,pred))

[[5249  199]
 [ 246 1014]]


In [26]:
pickle.dump(clf, open('pose_model_v1.p','wb'))